In [1]:
# import required stuff from the EMA Workbench
from ema_workbench.em_framework.model import FileModel, SingleReplication
from ema_workbench.util.ema_logging import method_logger

# import zip longest function
from itertools import zip_longest

# import numpy
import numpy as np

# import pandas
import pandas as pd

# import other required libraries
import subprocess, csv, os

# define a base class for interacting with Linny-R models
class BaseLinnyRModel(FileModel):
    
    # create an instance of this class
    def __init__(self, name, model_file = None):
        
        # define the name of the model object
        self.name = name
        
        # define the file of the model object and strip off '.lnr' part so Linny-R can find it
        self.model_file = model_file[:-4]
        
    # define a function for running an experiment
    def run_experiment(self, experiment):
        
        # create a csv input file readable by Linny-R from the experiment dict
        with open('exp.csv', 'w', newline = '') as file:
            
            # define the csv writer
            w = csv.writer(file, delimiter = ';')
            
            # write the variables names to the first row
            w.writerow(experiment.keys())
            
            # create a list of values where if item not already a list, make it a list (paramount for zip_longest function)
            values = [[i] if isinstance(i,list) == False else i for i in experiment.values()]
            
            # write the transposed values list to the next rows (works for timeseries and accounts for empty cells)
            w.writerows([list(filter(None,i)) for i in zip_longest(*values)])

        # execute Linny-R console using the experiment input file
        subprocess.call(['lrc.exe', self.model_file, 'exp.csv'])

        # read the csv output file from Linny-R and transpose it
        csv_reader = zip(*csv.reader(open(f'{self.model_file}_exp.csv'), delimiter=';'))

        # create empty dictionary for the results
        results = {}

        # loop through the rows containing variable name and values per time step ('variable','2','5','4')
        for row in csv_reader:

            # skip the first row containing the time step variable
            if row[0] == 'T':
                continue

            # create an empty list to store the values in
            values = []

            # loop through the items in each row
            for i in row:

                # try to append the item as a float to the value list
                try:
                    values.append(float(i))

                # if this rases an error, it is the first item containing the variable name
                except:
                    variable_name = i

            # append the dictionary with key = variable_name and value = values
            results[variable_name] = tuple(values)

        # delete the csv input file
        os.remove(f'exp.csv')

        # delete the output files (if things dont work out, have a look at the log file)
        os.remove(f'{self.model_file}_exp.csv')
        os.remove(f'{self.model_file}_exp.lp')
        os.remove(f'{self.model_file}_exp.log')

        # return the results
        return results

class LinnyRModel(SingleReplication, BaseLinnyRModel):
    pass

In [2]:
# this is a simple Linny-R model with 10 time steps
model = BaseLinnyRModel(name = 'smoothie_model', model_file = 'simple_model.lnr')

In [3]:
# create an experiment dictionary indentical to the one created by the EMA Workbench
# in this experiment the price of oranges and apples stays constant over time (10 steps)
ema_experiment = {'Oranges:Price': [30], 'Apples:Price': [80]}
ema_experiment

{'Oranges:Price': [30], 'Apples:Price': [80]}

In [4]:
# run experiment
# this Linny-R model only retrieves these two input variables as output to shows that it works
model.run_experiment(ema_experiment)

{'Apple price': (80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0),
 'Orange price': (30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0)}

In [5]:
# create another experiment dictionary
# this time the price changes over time
# first row of the csv input file must contain the default value for the variable (2 in this example)
# rows after that contain the values from time step one onwards
ema_experiment = {'Oranges:Price': [2,30,40,50,60,70,80,90,10,90,10], 
                  'Apples:Price': [2,30,40,50,60,70,80,90,10,90,10]}
ema_experiment

{'Oranges:Price': [2, 30, 40, 50, 60, 70, 80, 90, 10, 90, 10],
 'Apples:Price': [2, 30, 40, 50, 60, 70, 80, 90, 10, 90, 10]}

In [6]:
# running this also works
model.run_experiment(ema_experiment)

{'Apple price': (30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 10.0, 90.0, 10.0),
 'Orange price': (30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 10.0, 90.0, 10.0)}

In [7]:
# trying an EMA Workbench example
from ema_workbench import RealParameter, TimeSeriesOutcome, perform_experiments

model = LinnyRModel(name = 'Smoothie model', model_file = 'simple_model.lnr')

model.uncertainties = [RealParameter('Oranges:Price', 5, 20),
                       RealParameter('Apples:Price', 30, 40)]

model.outcomes = [TimeSeriesOutcome('Apples price'),
                  TimeSeriesOutcome('Orange price')]

In [8]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=100)

'Apples price'
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 84, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Anaconda\lib\site-packages\ema_workbench\util\ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "C:\Anaconda\lib\site-packages\ema_workbench\em_framework\model.py", line 340, in run_model
    self.outcomes_output = outputs
  File "C:\Anaconda\lib\site-packages\ema_workbench\em_framework\model.py", line 62, in outcomes_output
    data = [outputs[var] for var in outcome.variable_name]
  File "C:\Anaconda\lib\site-packages\ema_workbench\em_framework\model.py", line 62, in <listcomp>
    data = [outputs[var] for var in outcome.variable_name]
KeyError: 'Apples price'


EMAError: exception in run_model
Caused by: KeyError: 'Apples price'